# Job Performance Analytics

### Initialize Spark and Sonar Cassandra Session

In [ ]:
from sonar_driver.cassandra.session import SonarCassandraSession
from sonar_driver.spark.session import SonarSparkSession

from sonar_driver.spark import analytics
from sonar_driver.spark import job_analytics
from sonar_driver.spark import alloc_analytics
from sonar_driver.spark import visuals

import numpy as np
import pandas as pd

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType, IntegerType, TimestampType
from pyspark.sql.functions import col, lit, split, udf, explode, asc, desc

from datetime import datetime, timedelta

from bokeh.layouts import column
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import CustomJS, ColumnDataSource, HoverTool, DatetimeTickFormatter, DataRange, Range, FactorRange

In [ ]:
# List of Sonar nodes
hosts = ['rzsonar8']

# Path of Spark home
spark_home = '/g/g13/wang109/spark-2.3.1-bin-hadoop2.7'

scs = SonarCassandraSession(hosts=hosts)
sss = (
    SonarSparkSession(
        spark_home=spark_home,
        sonar_cassandra_session=scs
    )
)

### Read job data from Cassandra and store in Spark dataframe with appropriate column types

In [ ]:
# Format for reading into Cassandra
read_format = 'org.apache.spark.sql.cassandra'

# Provide keyspace and table name
keyspace, table = 'lcstaff_k', 'jobdata'

# Reading into Cassandra and selecting columns for job ID's, start times, end times, runtimes, nodes.
jobdf = (
    sss.spark_session.read
        .format(read_format)
        .options(keyspace=keyspace, table=table)
        .load()
        .select(['JobId', 'StartTime', 'scontrol'])
        .withColumn('JobId', col('JobId').cast(IntegerType()))
        .withColumn('StartTime', col('StartTime').cast(TimestampType()))
        .withColumn('EndTime', col('scontrol')['EndTime'].cast(TimestampType()))
        .withColumn('RunTime', col('scontrol')['RunTime'])
        .withColumn('NodeList', col('scontrol')['NodeList'])
        .drop('scontrol')
)

jobdf.show()

### Query jobs based on time range, nodes, users

In [ ]:
# Your given schema must match the keys of this schema. 
# The values of a schema are the strings of the column names of the corresponding keys.
schema = {
    'name': 'jobdata',
    'start': 'StartTime',
    'end': 'EndTime',
    'nodes': 'NodeList',
    'users': 'User'
}

# Time range must follow the timestamp format below.
time_range = ('2018-05-16T07:27:21', '2018-05-17T07:27:21')

# List of nodes must follow the format in the given table.
nodes = ['rztopaz', 'rzgenie36', 'rztronal[10-13]', 'rzalastor[10-15,20-24]']

# Query jobs based on time range and nodes.
queried_jobdf = job_analytics.query_jobs(jobdf, schema, time_range=time_range, nodes=nodes)
queried_jobdf.show()

### Calculate discrete derivatives based on window size and slide length

* 'column' refers to column (usually start times or end times) on which to calculate derivatives.
* 'window_size' refers to dX.
* 'slide_length' refers to the intervals at which derivatives are calculated.

In [ ]:
derivatives = analytics.discrete_derivatives(sparkdf=queried_jobdf, column=schema['end'], 
                                             window_size=300, slide_length=300)
derivatives.show()

### Plot discrete derivatives
* The 3 parameters after 'sparkdf' must match those passed into the 'discrete_derivatives' function.

In [ ]:
visuals.plot_derivatives(sparkdf=derivatives, column=schema['end'], window_size=300, slide_length=300)

### Calculate discrete integrals based on slide length
* 'start_column' refers to start times column name.
* 'end_column' refers to end times column name.
* 'slide_length' refers to the intervals at which integrals are calculated.

In [ ]:
integrals = analytics.discrete_integrals(sparkdf=queried_jobdf, start_column=schema['start'], 
                                         end_column=schema['end'], slide_length=10)
integrals.show()

### Plot discrete integrals
* 'slide_length' must match the one passed into the 'discrete_derivatives' function.

In [ ]:
visuals.plot_integrals(sparkdf=integrals, slide_length=10)

### Convert to Pandas dataframe for convenience

In [ ]:
qdf = queried_jobdf.sort('StartTime').toPandas()

### Gantt chart of jobs unpooled
* 'hist_var' refers to the column which contains the variable for the histogram.
* 'hist_grouped' if True, each bin in the histogram will be a range of values; if False, each bin will be an individual value.
* 'gantt_pooled' if True, resulting Gantt chart will pool objects; if False, each object will reside on its own horizontal line.
* 'unit' refers to unit of timestamps in sparkdf.
* 'df' is a Pandas dataframe which can be optionally inputed to reduce redundant operations.

In [ ]:
visuals.plot_hist_gantt(queried_jobdf, start_column='StartTime', end_column='EndTime', df=qdf,
                        hist_var='RunTime', hist_grouped=True, gantt_pooled=False, unit='s')

### Gantt chart of jobs pooled

In [ ]:
visuals.plot_hist_gantt(queried_jobdf, start_column='StartTime', end_column='EndTime', df=qdf,
                        hist_var='RunTime', hist_grouped=True, gantt_pooled=True, unit='s')

### Original dataframe is unaltered

In [ ]:
jobdf.show()

# Memory Allocations Analytics

### Read file of allocations

In [ ]:
# Format for reading txt file
read_format = 'com.databricks.spark.csv'

# File of allocations
allocs_file = 'allocs_file.txt'

# Reading into text and selecting columns for address, sizes, alloc times, free times.
allocdf = (
    sss.spark_session.read
        .format(read_format)
        .option('delimiter', ',')
        .option('header', 'false')
        .load(allocs_file)
        .toDF('address', 'size', 'alloc_time', 'free_time')
        .withColumn('size', col('size').cast(DoubleType()).cast(IntegerType()))
        .withColumn('alloc_time', col('alloc_time').cast(DoubleType()))
        .withColumn('free_time', col('free_time').cast(DoubleType()))
)

# Cleans allocation dataframe to account for allocs which were not deallocated.
max_free_time = allocdf.agg({"free_time": "max"}).collect()[0][0]
set_free_time = udf(lambda t: max_free_time if t == 0 else t, DoubleType())
allocdf = allocdf.withColumn('free_time', set_free_time('free_time'))

In [ ]:
allocdf.show()

### Calculate metrics of unpooled and pooled allocations
* 'start_column' refers to start time column name
* 'end_column' refers to end time column name
* 'var' refers to variable on which to calculate metrics (size in this case)

In [ ]:
start_column, end_column, var = 'alloc_time', 'free_time', 'size'

max_memory_unpooled = alloc_analytics.max_memory_unpooled(allocdf, start_column, end_column, var)
max_memory_pooled = alloc_analytics.max_memory_pooled(allocdf, start_column, end_column, var)
total_bytesecs_unpooled = alloc_analytics.max_memory_pooled(allocdf, start_column, end_column, var)
total_bytesecs_pooled = alloc_analytics.total_bytesecs_pooled(allocdf, start_column, end_column, var)

print('max_memory_unpooled:', max_memory_unpooled)
print('max_memory_pooled:', max_memory_pooled)
print('total_bytesecs_unpooled:', total_bytesecs_unpooled)
print('total_bytesecs_pooled:', total_bytesecs_pooled)

### Calculate minimum number of pools for each unique allocation size

In [ ]:
pools = sorted(alloc_analytics.pool_counts(allocdf, start_column, end_column, var), key=lambda p: -p['count'])
pools

### Convert to Pandas dataframe for convenience

In [ ]:
adf = allocdf.toPandas()

### Unpooled Gantt chart of allocations

In [ ]:
visuals.plot_hist_gantt(allocdf, start_column='alloc_time', end_column='free_time', df=adf,
                        hist_var='size', hist_grouped=False, gantt_pooled=False, unit='ns')

### Pooled Gantt chart of allocations

In [ ]:
visuals.plot_hist_gantt(allocdf, start_column='alloc_time', end_column='free_time', df=adf,
                        hist_var='size', hist_grouped=False, gantt_pooled=True, unit='ns')

### Original dataframe is unaltered

In [ ]:
allocdf.show()